In [19]:
import os
import os.path
import shutil
import tensorflow as tf
import numpy as np

In [15]:
LOGDIR = "/tmp/mnist_tutorial/"
LABELS = os.path.join(os.getcwd(), "labels_1024.tsv")
SPRITES = os.path.join(os.getcwd(), "sprite_1024.png")
### MNIST EMBEDDINGS ###
mnist = tf.contrib.learn.datasets.mnist.read_data_sets(train_dir=LOGDIR + "data", one_hot=True)
### Get a sprite and labels file for the embedding projector ###

if not (os.path.isfile(LABELS) and os.path.isfile(SPRITES)):
  print("Necessary data files were not found. Run this command from inside the "
    "repo provided at "
    "https://github.com/dandelionmane/tf-dev-summit-tensorboard-tutorial.")
  exit(1)


# shutil.copyfile(LABELS, os.path.join(LOGDIR, LABELS))
# shutil.copyfile(SPRITES, os.path.join(LOGDIR, SPRITES))


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/mnist_tutorial/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/mnist_tutorial/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/mnist_tutorial/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/mnist_tutorial/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Plea

In [29]:

from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.examples.tutorials.mnist import mnist

In [22]:
# Define a simple convolutional layer
def conv_layer(input, channels_in, channels_out):
    w = tf.Variable(tf.zeros([5,5,channels_in, channels_out]))
    b = tf.Variable(tf.zeros([channels_out]))
    conv = tf.nn.conv2d(input, w, [1,1,1,1], padding="SAME")
    act = tf.nn.relu(conv + b)
    return act

# And a fully connected layer
def fc_layer(input, channels_in, channels_out):
    w = tf.Variable(tf.zeros([channels_in, channels_out]))
    b = tf.Variable(tf.zeros([channels_out]))
    act = tf.nn.relu(tf.matmul(input, w) + b)
    return act

In [23]:
# Feed Forword Step

## Setup placeholders, and reshape the data
x = tf.placeholder(tf.float32, shape=[None, 784])
y = tf.placeholder(tf.float32, shape=[None, 10])
x_image = tf.reshape(x, [-1, 28, 28, 1])

In [24]:
# Create the network

conv1 = conv_layer(x_image, 1, 32)
pool1 = tf.nn.max_pool(conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")

conv2 = conv_layer(pool1, 32, 64)
pool2 = tf.nn.max_pool(conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
flattened = tf.reshape(pool2, shape=[-1, 7*7*64])

fc1 = fc_layer(flattened, 7*7*64, 1024)
logits = fc_layer(fc1, 1024, 10)

In [25]:
# Loss and Training

## Compute cross entropy as our loss function
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y))

## Use an ADAM optimizer to train the network
train_step = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cross_entropy)

## Compute the accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))



In [28]:
# Train the model

## Initialize all the variables
sess = tf.InteractiveSession()

sess.run(tf.global_variables_initializer())

# Train for 2000 steps
for i in range(2000):
    batch = mnist.train.next_batch(100)
    
    #Occasionally report the accuracy
    if i % 500 == 0:
        [train_accuracy] = sess.run([accuracy], feed_dict={x:batch[0], y:batch[1]})
        print("step %d, training_accuracy %g" %(i, train_accuracy))
        
    # Run the training step
    sess.run(train_step, feed_dict={x:batch[0], y:batch[1]})

/anaconda3/envs/tfdeeplearning/lib/python3.5/site-packages/tensorflow/python/client/session.py:1645: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


AttributeError: module 'tensorflow.examples.tutorials.mnist' has no attribute 'train'

# Visualizing the Graph

In [30]:
#tf.summary.FileWriter(n):
#    """a python class that writes data for TensorBoard"""
    
writer = tf.summary.FileWriter("/tmp/mnist_demo/1")
writer.add_graph(sess.graph)
    